In [ ]:
import sys
!{sys.executable} -m pip uninstall ldaca -y

In [ ]:
import sys
!{sys.executable} -m pip install python-dotenv>0.19.2
!{sys.executable} -m pip install ldaca@git+https://github.com/Language-Research-Technology/ldaca-py.git@v0.0.2.dev

In [6]:
import os
from dotenv import load_dotenv
from ldaca.ldaca import LDaCA
URL = 'https://oni-demo.text-commons.org/api'
COLLECTION_ID = 'arcp://name,sydney-speaks/corpus/root'
load_dotenv('vars.env') 
TOKEN = os.getenv('API_KEY') 

In [7]:
ldaca = LDaCA(url=URL, token=TOKEN)

In [ ]:
# Saves the metadata in the data_dir
ldaca.get_collection(collection=COLLECTION_ID, collection_type='Collection', data_dir='data')

In [ ]:
# Gets the collection member 3 and print the Id
members = ldaca.get_members_of_collection()
memberId = ldaca.collection_members[3]['crateId']
memberId

In [ ]:
# Finds files with a file picker and store data and metadata
my_other_file_picker = lambda f : f if 'OrthographicTranscription' in f['@type'] else None
ldaca.store_data(sub_collection=memberId, entity_type='DialogueText', file_picker=my_other_file_picker)

In [ ]:
ldaca.ldaca_files_path

In [14]:
data = ldaca.pandas_dataframe

In [ ]:
data